In [5]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sqlalchemy import create_engine
import matplotlib as mpl
import numpy as np
import seaborn as sns
from scipy import stats
from sklearn import preprocessing
%matplotlib inline

In [7]:
path = r"XXXX-XXXX"
con = sqlite3.connect(path)
cur = con.cursor()
goalkeeper = pd.read_sql_query("SELECT *  FROM avgPlayerStatistics AS ps WHERE ps.player_position = 'G' ;", con,)

In [8]:
pd.set_option('display.max_columns', None)
goalkeeper.head()

,index,assists,blocks,commited_fouls,commited_penalty,conceded,drawn_fouls,dribble_attempts,dribble_past,dribble_success,fixture_id,interceptions,key_passes,minutes,missed_penalty,number,offside,on_goal_shots,pass_accuracy,player_id,player_position,rating,red,saved_penalty,saves,scored_penalty,timestamp,total_duels,total_goals,total_passes,total_shots,total_tackels,won_duels,won_penalty,yellow
0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,185545,1.0,0.0,90.0,0.0,1.0,0.0,0.0,75.0,87166,G,6.5,0.0,0.0,1.0,0.0,1438972200,2.0,0.0,18.0,0.0,0.0,1.0,0.0,0.0
1,11,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,185545,0.0,0.0,90.0,0.0,16.0,0.0,0.0,43.0,1800,G,7.0,0.0,0.0,2.0,0.0,1438972200,1.0,0.0,7.0,0.0,0.0,1.0,0.0,0.0
2,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,192297,0.0,0.0,90.0,0.0,20.0,0.0,0.0,52.0,884,G,7.4,0.0,0.0,4.0,0.0,1439034300,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0
3,33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,192297,0.0,0.0,90.0,0.0,13.0,0.0,0.0,93.0,160,G,6.6,0.0,0.0,1.0,0.0,1439034300,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0
4,44,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,192298,0.0,0.0,90.0,0.0,1.0,0.0,0.0,90.0,18859,G,6.5,0.0,0.0,2.0,0.0,1439042400,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0


In [39]:
df = goalkeeper[['fixture_id','player_id','saves','conceded','pass_accuracy','total_passes','won_duels','minutes']]

#display(df)
df.head()




,fixture_id,player_id,saves,conceded,pass_accuracy,total_passes,won_duels,minutes
0,185545,87166,1.0,1.0,75.0,18.0,1.0,90.0
1,185545,1800,2.0,0.0,43.0,7.0,1.0,90.0
2,192297,884,4.0,0.0,52.0,19.0,0.0,90.0
3,192297,160,1.0,1.0,93.0,42.0,0.0,90.0
4,192298,18859,2.0,1.0,90.0,10.0,0.0,90.0


In [11]:
for key in df.keys(): # check for equality of dataframes
    print(key,":", defender[key].equals(df[key]))

fixture_id : True
player_id : True
saves : True
conceded : True
pass_accuracy : True
total_passes : True
won_duels : True
minutes : True


In [12]:
# NORMALIZE ALLE AUßER FIXTURE_ID

x = df.to_numpy() #returns a numpy array
x1 = x[:, 2:] # every column from colum (player_id)


robust_scaler = preprocessing.RobustScaler()
x_scaled = robust_scaler.fit_transform(x1)
x_scaled = np.concatenate(( x[:,[0,1]], x_scaled), axis=1)
final = pd.DataFrame(x_scaled, columns=df.columns)
columns = df.keys()
for i in range(2): # fixture_id bis timestamp zu int
    final[columns[i]] = pd.to_numeric(final[columns[i]], downcast='integer')
display(final)

,fixture_id,player_id,saves,conceded,pass_accuracy,total_passes,won_duels,minutes
0,185545,87166,-1.285714,-0.2,0.794872,0.095238,1.333333,0.0
1,185545,1800,-0.571429,-1.2,-0.572650,-1.214286,1.333333,0.0
2,192297,884,0.857143,-1.2,-0.188034,0.214286,-0.333333,0.0
3,192297,160,-1.285714,-0.2,1.564103,2.952381,-0.333333,0.0
4,192298,18859,-0.571429,-0.2,1.435897,-0.857143,-0.333333,0.0
...,...,...,...,...,...,...,...,...
28402,705681,24785,-1.000000,0.2,-1.495726,1.047619,-0.333333,0.0
28403,705637,30393,-0.142857,-0.4,-1.982906,1.023810,0.333333,0.0
28404,705637,21079,-0.714286,-0.2,-0.444444,0.000000,0.666667,0.0
28405,705683,46811,-0.428571,0.8,-1.589744,1.309524,1.000000,0.0


In [13]:
#NORMALIZE ALL FOR COMPARISON

x = df.to_numpy() #returns a numpy array
robust_scaler = preprocessing.RobustScaler()
x_scaled = robust_scaler.fit_transform(x)
all_norm = pd.DataFrame(x_scaled, columns=df.columns)

In [14]:
#TESTING FINALDATASET FOR FIXTURE ORDER

columns = df.keys()
print(columns[0]) #check für index 0,1,2 ist per hand passiert
x = columns[0]
final[x] = pd.to_numeric(final[x], downcast='integer')
df[x] = pd.to_numeric(df[x], downcast='integer')
print(df[x].equals(final[x]))  # Returns True)
print(type(columns))

#TEST 
# haben fixture_id bis timestamp gleiche Reihenfolge
# wenn ja sind in richtiger reiheinfolge konkateniert

fixture_id
True
<class 'pandas.core.indexes.base.Index'>


C:\Users\Anwender\AppData\Local\Temp/ipykernel_8700/3261938473.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[x] = pd.to_numeric(df[x], downcast='integer')


In [15]:
for key in df.keys():
    print(key,":", all_norm[key].equals(final[key]))

fixture_id : False
player_id : False
saves : True
conceded : True
pass_accuracy : True
total_passes : True
won_duels : True
minutes : True


In [17]:
weights = [1,1,0.4,0.4,0.05,0.05,0.05,0.05]
print(len(weights))
final_mul_weights = final.mul(weights)
final_mul_weights['fixture_id'] = pd.to_numeric(final_mul_weights['fixture_id'], downcast='integer')
final_mul_weights['player_id'] = pd.to_numeric(final_mul_weights['player_id'], downcast='integer')  #um equals true zu erreichen müssen fixture id gleichen typ haben
print("fixture_id:", final['fixture_id'].equals(final_mul_weights['fixture_id']))
print("player_id:", final['player_id'].equals(final_mul_weights['player_id']))
final_mul_weights.head()

8
fixture_id: True
player_id: True


,fixture_id,player_id,saves,conceded,pass_accuracy,total_passes,won_duels,minutes
0,185545,87166,-0.514286,-0.08,0.039744,0.004762,0.066667,0.0
1,185545,1800,-0.228571,-0.48,-0.028632,-0.060714,0.066667,0.0
2,192297,884,0.342857,-0.48,-0.009402,0.010714,-0.016667,0.0
3,192297,160,-0.514286,-0.08,0.078205,0.147619,-0.016667,0.0
4,192298,18859,-0.228571,-0.08,0.071795,-0.042857,-0.016667,0.0


In [18]:
fixtures = final_mul_weights.pop('fixture_id')
players = final_mul_weights.pop('player_id')


In [19]:
fixtures.head()

0    185545
1    185545
2    192297
3    192297
4    192298
Name: fixture_id, dtype: int32

In [20]:
test = final_mul_weights.sum(axis=1)
display(test)
test.head()
player_rating = pd.Series(test,
              name="g_rating")
player_rating = player_rating.to_frame()
print(type(player_rating))

0       -0.483114
1       -0.731252
2       -0.152497
3       -0.385128
4       -0.296300
           ...   
28402   -0.359072
28403   -0.248431
28404   -0.354603
28405    0.184560
28406   -1.286807
Length: 28407, dtype: float64

<class 'pandas.core.frame.DataFrame'>


In [21]:
player_rating.insert(0, 'fixture_id', fixtures)
player_rating.insert(1, 'player_id', players)

In [22]:
#TESTING FINALDATASET FOR FIXTURE ORDER

columns = df.keys()
print(columns[0]) #check für index 0,1,2 ist per hand passiert
x = columns[0]
player_rating[x] = pd.to_numeric(player_rating[x], downcast='integer')
df[x] = pd.to_numeric(df[x], downcast='integer')
print(df[x].equals(player_rating[x]))  # Returns True)
print(type(columns))

#TEST 
# haben fixture_id bis timestamp gleiche Reihenfolge
# wenn ja sind in richtiger reiheinfolge konkateniert

fixture_id
True
<class 'pandas.core.indexes.base.Index'>


C:\Users\Anwender\AppData\Local\Temp/ipykernel_8700/2261603312.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[x] = pd.to_numeric(df[x], downcast='integer')


In [23]:
player_rating.describe()
pd.get_option("display.max_rows")
pd.set_option("display.max_rows", 20)

In [24]:
engine = create_engine(r'XXXX-XXXX', echo=False)
player_rating.to_sql('goalkeeperRating', con=engine, if_exists='append') #if_exists='append'

In [36]:
test = pd.read_sql_query("SELECT DISTINCT player_id, g_rating FROM goalkeeperRating WHERE player_id IN (SELECT player_id FROM goalkeeperRating ORDER BY g_rating DESC LIMIT 20);", con)
display(test)

,player_id,g_rating
0,19491,2.387949
1,19491,0.937216
2,19491,0.364923
3,19491,0.279966
4,2932,2.179341
...,...,...
381,2932,-0.860598
382,47326,-3.736087
383,2932,-0.409603
384,2932,-0.411184


In [37]:
test = pd.read_sql_query("SELECT  * FROM lineups WHERE player_id = 19491 AND player_position = 'G';", con)
display(test)

,fixture_id,team_id,timestamp,player_id,player_name,player_number,player_position,grid
0,192379,35,1445090400,19491,A. Federici,23,G,None
1,192405,35,1446393600,19491,A. Federici,23,G,None
2,192407,35,1446900300,19491,A. Federici,23,G,None
3,192420,35,1448118000,19491,A. Federici,23,G,None
4,192427,35,1448722800,19491,A. Federici,23,G,None
5,192676,35,1463511600,19491,A. Federici,23,G,None
6,17569,35,1480256100,19491,A. Federici,23,G,1:1
7,17583,35,1479567600,19491,A. Federici,23,G,1:1


In [38]:
test = pd.read_sql_query("SELECT * FROM goalkeeperRating", con)
display(test)

,index,fixture_id,player_id,g_rating
0,0,185545,87166,-0.483114
1,1,185545,1800,-0.731252
2,2,192297,884,-0.152497
3,3,192297,160,-0.385128
4,4,192298,18859,-0.296300
...,...,...,...,...
28402,28402,705681,24785,-0.359072
28403,28403,705637,30393,-0.248431
28404,28404,705637,21079,-0.354603
28405,28405,705683,46811,0.184560
